In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
import logging
import os
# Set both HTTP and HTTPS proxies
os.environ["HTTP_PROXY"]  = "http://proxy.isoad.isogmbh.de:81"
os.environ["HTTPS_PROXY"] = "http://proxy.isoad.isogmbh.de:81"

# 1. EINMALIGER DOWNLOAD (Außerhalb der Schleife)
tickers = ['AAPL', 'SPY']
df_raw_multi = yf.download(tickers, start='2021-06-01', end='2025-01-01')

# 2. BASIS-BERECHNUNG (Log-Returns & Volumen)
# Wir berechnen hier nur die Roh-Features, die keine Roll-Fenster nutzen
df_close_log = np.log(df_raw_multi['Close'] / df_raw_multi['Close'].shift(1))
df_vol_log = np.log(df_raw_multi['Volume']).diff()

# 3. DIE SCHLEIFE
test_dates = df_close_log['2024-01-01':].index
portfolio_value = 10000.0
current_pos = 0
THRESHOLD = 0.000553
FEE = 0.0005

for i, today in enumerate(test_dates):
    # --- DATEN-FENSTER FÜR HEUTE ERSTELLEN ---
    df_list = []
    for t in tickers:
        # Nur Daten bis 'today'
        mask = df_close_log.index <= today
        
        # Preis-Kanäle
        df_list.append(pd.DataFrame({
            'ds': df_close_log.index[mask],
            'unique_id': f'{t}_price',
            'y': df_close_log[t][mask]
        }))
        # Volumen-Kanäle
        df_list.append(pd.DataFrame({
            'ds': df_vol_log.index[mask],
            'unique_id': f'{t}_vol',
            'y': df_vol_log[t][mask]
        }))

    df_step = pd.concat(df_list).dropna()
    
    # Momentum-Target berechnen (NUR auf den Daten bis heute!)
    df_step['y_rolling'] = df_step.groupby('unique_id')['y'].transform(lambda x: x - x.rolling(5).mean())
    df_step = df_step.dropna().rename(columns={'y': 'y_raw', 'y_rolling': 'y'})
    
    # --- MODELL-TRAINING (Dein Stable Setup) ---
    model = PatchTST(
        h=7,
        input_size=10,
        patch_len=2,
        stride=1,
        max_steps=300,
        learning_rate=5e-5,
        early_stop_patience_steps=3,
        val_check_steps=50,
        optimizer_kwargs={'weight_decay': 0.01},
        accelerator='gpu',
        devices=1
    )

    nf = NeuralForecast(models=[model], freq='D')
    nf.fit(df=df_step, val_size=20)
    # Prediction für den nächsten Tag
    forecast = nf.predict(df=df_step)
    # Wir nehmen den ersten Tag des Horizonts (t+1)
    pred_momentum = forecast.query("unique_id == 'AAPL_price'").iloc[0]['PatchTST']
    
    # Signal Logik
    signal = 0
    if pred_momentum > THRESHOLD: signal = 1
    elif pred_momentum < -THRESHOLD: signal = -1
    
    # Abrechnung (am nächsten Handelstag)
    if i + 1 < len(test_dates):
        next_day = test_dates[i+1]
        actual_ret = df_raw.loc[next_day, 'y_raw']
        
        # Kosten nur bei Signalwechsel
        cost = FEE if signal != current_pos else 0
        daily_profit = (signal * actual_ret) - cost
        portfolio_value *= np.exp(daily_profit)
        current_pos = signal
        
        # Output & Log
        print(f"{today.date()} | Pred: {pred_momentum:.5f} | Signal: {signal} | Port: {portfolio_value:.2f}€")
        with open(log_file, 'a') as f:
            f.write(f"{today.date()},{pred_momentum},{signal},{actual_ret},{portfolio_value}\n")

    #return log_file

# Start den Prozess
#run_stable_walk_forward()

[*********************100%***********************]  2 of 2 completed
Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/neuralforecast/common/_base_model.py:632: UserWarning: ignoring optimizer_kwargs as the optimizer is not specified
  warnings.warn(

  | Name         | Type              | Params | Mode  | FLOPs
-------------------------------------------------------------------
0 | loss         | MAE               | 0      | train | 0    
1 | padder_train | ConstantPad1d     | 0      | train | 0    
2 | scaler       | TemporalNorm      | 0      | train | 0    
3 | model        | PatchTST_backbone | 408 K  | t

/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 15.67it/s, v_num=6, train_loss_step=0.100, train_loss_epoch=0.100, valid_loss=0.116]  

`Trainer.fit` stopped: `max_steps=300` reached.


Epoch 299: 100%|██████████| 1/1 [00:00<00:00, 14.97it/s, v_num=6, train_loss_step=0.100, train_loss_epoch=0.100, valid_loss=0.116]


Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aidev/patchtst_timeseries/.venv/lib/python3.13/site-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 66.43it/s]


NameError: name 'df_raw' is not defined